In [164]:
# data frames/structures
import pandas as pd
import numpy as np

# pytorch - neural network library
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
# custom class used to define the LossTriangleClassifier network
from model.LossTriangleClassifier import LossTriangleClassifier

In [3]:
# Triangle class imported from (future) rocky package
from triangle import Triangle

In [4]:
# methods for working with excel files
import openpyxl
from openpyxl.utils import range_to_tuple

In [5]:
# pre-fit/saved triangle model
model_file = r"C:\Users\aweaver\OneDrive - The Cincinnati Insurance Company\rocky\inc_cum_tri.torch"
# model_file = r"C:\Users\AndyW\OneDrive\Documents\inc_cum_tri.torch"

In [206]:
rpt_cum = Triangle.from_clipboard()
rpt_cum.tri

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228,240
Acc. Year,,,,,,,,,,,,,,,,,,,,
2003-01-01,93395.0,113749.0,122853.0,134290.0,132177.0,125464.0,124303.0,125682.0,122249.0,123009.0,124573.0,124000.0,124265.0,124623.0,124699.0,125806.0,125827.0,125817.0,125810.0,125760.0
2004-01-01,79155.0,105322.0,123113.0,116625.0,111198.0,109560.0,108327.0,105876.0,106111.0,108108.0,107198.0,107346.0,107064.0,107257.0,107297.0,107378.0,107310.0,107300.0,107298.0,NaN
2005-01-01,101203.0,126675.0,134459.0,128810.0,115779.0,115976.0,111804.0,112924.0,115444.0,116545.0,116871.0,116823.0,117259.0,115368.0,115453.0,115137.0,115053.0,114990.0,NaN,NaN
2006-01-01,109538.0,124758.0,134715.0,129259.0,128678.0,122774.0,121837.0,120994.0,120507.0,120086.0,120267.0,120794.0,121341.0,121392.0,121421.0,122676.0,122475.0,NaN,NaN,NaN
2007-01-01,126907.0,148870.0,160861.0,148592.0,135756.0,129914.0,128133.0,126956.0,128915.0,127491.0,128848.0,126770.0,127233.0,127778.0,127606.0,127182.0,NaN,NaN,NaN,NaN
2008-01-01,134648.0,162687.0,164788.0,156025.0,144477.0,137472.0,134865.0,133124.0,133161.0,133233.0,132477.0,132427.0,132831.0,134067.0,134018.0,NaN,NaN,NaN,NaN,NaN
2009-01-01,114412.0,133818.0,130351.0,124827.0,116407.0,115273.0,113364.0,112100.0,112874.0,113785.0,113550.0,114575.0,114629.0,116903.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-01,111153.0,125651.0,126018.0,116053.0,116077.0,119247.0,117522.0,118201.0,117175.0,117618.0,118415.0,119200.0,119456.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-01,111116.0,121729.0,126488.0,130759.0,128368.0,126992.0,127003.0,127437.0,126385.0,128032.0,126736.0,126575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### preprocess data:
1. Remove AY < 2008 that are all blank
2. convert to 10x10 triangle
3. reshape as if it were a standard yearly triangle (just remove the extra columns so that it is a 10x10 triangle)

In [151]:
rpt_cum.tri

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228,240
Acc. Year,,,,,,,,,,,,,,,,,,,,
2003-01-01,93395.0,113749.0,122853.0,134290.0,132177.0,125464.0,124303.0,125682.0,122249.0,123009.0,124573.0,124000.0,124265.0,124623.0,124699.0,125806.0,125827.0,125817.0,125810.0,125760.0
2004-01-01,79155.0,105322.0,123113.0,116625.0,111198.0,109560.0,108327.0,105876.0,106111.0,108108.0,107198.0,107346.0,107064.0,107257.0,107297.0,107378.0,107310.0,107300.0,107298.0,NaN
2005-01-01,101203.0,126675.0,134459.0,128810.0,115779.0,115976.0,111804.0,112924.0,115444.0,116545.0,116871.0,116823.0,117259.0,115368.0,115453.0,115137.0,115053.0,114990.0,NaN,NaN
2006-01-01,109538.0,124758.0,134715.0,129259.0,128678.0,122774.0,121837.0,120994.0,120507.0,120086.0,120267.0,120794.0,121341.0,121392.0,121421.0,122676.0,122475.0,NaN,NaN,NaN
2007-01-01,126907.0,148870.0,160861.0,148592.0,135756.0,129914.0,128133.0,126956.0,128915.0,127491.0,128848.0,126770.0,127233.0,127778.0,127606.0,127182.0,NaN,NaN,NaN,NaN
2008-01-01,134648.0,162687.0,164788.0,156025.0,144477.0,137472.0,134865.0,133124.0,133161.0,133233.0,132477.0,132427.0,132831.0,134067.0,134018.0,NaN,NaN,NaN,NaN,NaN
2009-01-01,114412.0,133818.0,130351.0,124827.0,116407.0,115273.0,113364.0,112100.0,112874.0,113785.0,113550.0,114575.0,114629.0,116903.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-01,111153.0,125651.0,126018.0,116053.0,116077.0,119247.0,117522.0,118201.0,117175.0,117618.0,118415.0,119200.0,119456.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-01,111116.0,121729.0,126488.0,130759.0,128368.0,126992.0,127003.0,127437.0,126385.0,128032.0,126736.0,126575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
rpt_cum.tri['12']

Acc. Year
2003-01-01     93395.0
2004-01-01     79155.0
2005-01-01    101203.0
2006-01-01    109538.0
2007-01-01    126907.0
2008-01-01    134648.0
2009-01-01    114412.0
2010-01-01    111153.0
2011-01-01    111116.0
2012-01-01     95181.0
2013-01-01    100388.0
2014-01-01    114204.0
2015-01-01    117477.0
2016-01-01    110972.0
2017-01-01    118706.0
2018-01-01    125459.0
2019-01-01    121426.0
2020-01-01    100776.0
2021-01-01     96252.0
2022-01-01    125749.0
Name: 12, dtype: float64

In [205]:
dat = rpt_cum.prep_for_cnn()
dat

,12,24,36,48,60,72,84,96,108,120
Acc. Year,,,,,,,,,,
2013-01-01,-1.253111,-1.711567,-1.683038,-1.644409,-1.375716,-1.666310,-1.539476,-1.140998,-1.0,0.0
2014-01-01,0.104461,0.015676,-0.400086,-0.500216,-0.661216,-0.498790,-0.231560,-0.153099,1.0,0.0
2015-01-01,0.426069,-0.046470,-0.239919,-0.320767,-0.251327,0.470317,0.821838,1.294096,0.0,0.0
2016-01-01,-0.213118,-0.065598,-0.382445,-0.508247,-0.130433,0.449097,0.949198,0.000000,0.0,0.0
2017-01-01,0.546832,0.800044,0.628245,0.291190,0.632725,1.245686,0.000000,0.000000,0.0,0.0
2018-01-01,1.210387,1.707805,1.702884,1.310085,1.785967,0.000000,0.000000,0.000000,0.0,0.0
2019-01-01,0.814101,1.015309,1.035387,1.372364,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2020-01-01,-1.214986,-0.909328,-0.661028,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2021-01-01,-1.659518,-0.805871,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [159]:
# load CNN model
model = LossTriangleClassifier(torch.Size([1,10,10]), 
                               num_classes=2,
                               num_conv_layers=5,
                               base_conv_nodes=256,
                               kernel_size=(2, 2),
                               stride=(1, 1),
                               padding=(1, 1),
                               linear_nodes=[1024, 512, 256, 128],
                               linear_dropout=[0.4, 0.3, 0.2, 0.1],
                               relu_neg_slope=0.1)
model.to(torch.device('cpu'))
model.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))


<All keys matched successfully>

In [204]:
%run triangle.py
tri = Triangle.from_clipboard()
tri.tri.index.name = 'ay'
tri.tri.fillna('')

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228,240
ay,,,,,,,,,,,,,,,,,,,,
2003-01-01,16700.0,1884.0,-372.0,11825.0,-72.0,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,-12.0,0.0,0.0,0.0,0.0
2004-01-01,7815.0,1031.0,-176.0,5.0,-35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2005-01-01,9877.0,1233.0,-454.0,70.0,-8.0,16.0,-20.0,707.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2006-01-01,10467.0,525.0,31.0,-89.0,-2.0,-3.0,-1.0,1.0,2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2007-01-01,18571.0,557.0,49.0,-99.0,2.0,-20.0,466.0,-36.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2008-01-01,19466.0,-142.0,-17.0,-28.0,-22.0,-7.0,-1.0,22.0,0.0,63.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-01-01,19232.0,293.0,-47.0,55.0,11.0,-308.0,0.0,-1.0,0.0,-2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01,21363.0,473.0,1073.0,22.0,262.0,17.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01,21680.0,-4.0,372.0,216.0,-51.0,-127.0,0.0,222.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [177]:
tri.prep_for_cnn()


,12,24,36,48,60,72,84,96,108,120
Acc. Year,,,,,,,,,,
2013-01-01,-1.253111,-0.352278,0.117362,-1.396095,0.552502,-2.289009,0.492021,0.915397,0.686993,3.000000
2014-01-01,0.104461,0.113386,-0.108475,-0.454096,-2.530103,-0.642175,0.382339,-1.674941,2.825866,-0.333333
2015-01-01,0.426069,-0.227459,0.255987,-0.215167,-1.125492,1.755759,-0.381214,2.439926,-0.439107,-0.333333
2016-01-01,-0.213118,0.313423,0.036878,-0.519912,0.888167,0.972357,2.772838,-0.240055,-0.439107,-0.333333
2017-01-01,0.546832,0.477183,0.558753,-0.115708,0.407616,0.544570,-0.544331,-0.240055,-0.439107,-0.333333
2018-01-01,1.210387,0.765591,1.131534,0.873815,0.939959,-0.068300,-0.544331,-0.240055,-0.439107,-0.333333
2019-01-01,0.814101,0.449417,0.957618,2.563305,0.216838,-0.068300,-0.544331,-0.240055,-0.439107,-0.333333
2020-01-01,-1.214986,0.382280,0.748411,-0.245380,0.216838,-0.068300,-0.544331,-0.240055,-0.439107,-0.333333
2021-01-01,-1.659518,0.870208,-1.849033,-0.245380,0.216838,-0.068300,-0.544331,-0.240055,-0.439107,-0.333333


In [178]:
data = DataLoader(tri.prep_for_cnn().values, batch_size=1)
model.eval()
with torch.no_grad():
    # for inputs in data:
    #     print(inputs)
    inputs = torch.from_numpy(data.dataset.reshape(1,10,10))
    pred = model(inputs.float().unsqueeze(0))

In [187]:
torch.argmax(pred, dim=1).cpu().item()

0